# Class 4 - Hybrid LCA

In this class, we will learn about supply use tables, and input output tables. We will also do a toy hybrid LCA.

Before getting started, make sure you have upgrade the Brightway2 packages. You should have at least the following:

In [1]:
import bw2data, bw2calc, bw2io
print("BW2 data:", bw2data.__version__)
print("BW2 calc:", bw2calc.__version__)
print("BW2 io:", bw2io.__version__)

BW2 data: (2, 0, 'dev5')
BW2 calc: (1, 1, 'dev2')
BW2 io: (0, 2, 'dev5')


We will need the latest version of the data migrations to match EXIOBASE biosphere flows to ecoinvent biosphere flows:

In [3]:
create_core_migrations()

Now import the necessary libraries:

In [1]:
from brightway2 import *
from bw2io.importers.exiobase import Exiobase22Importer
import numpy as np
import os
import pyprind

Create a new project for this class:

In [2]:
if 'Class 4' not in projects:
    projects.current = "Class 1"
    projects.copy_project("Class 4")

projects.current = "Class 4"

# Import EXIOBASE 2.2

Now we need to download the industry by industry table from version 2.2 of exiobase. You can get it from the following link. Note that you will have to register an account if this is the first time you use this database: http://www.exiobase.eu/index.php/data-download/exiobase2-year-2007-full-data-set/78-mriot-ixi-fpa-coefficient-version2-2-2/file

Extract the downloaded file, and adjust the following. Windows users might need something like:

    fp = "C:\\Users\\<your name>\\Downloads\\mrIOT_IxI_fpa_coefficient_version2.2.2"

In [3]:
fp = "/Users/cmutel/Downloads/mrIOT_IxI_fpa_coefficient_version2.2.2"

assert os.path.exists(fp), "Please adjust your filepath, the provided one doesn't work"

We can now import the exiobase database. This will take a while, so go ahead and get started.

Why is this so slow compared to ecoinvent, for example? The answer lies in the density of the technosphere matrix. Exiobase, and IO tables in general, use comprehensive data from surveys and national customs, so they will get data on things that normal people would never even think of. For example, how much rice from Thailand is required to produce one euro of steel in Germany?

In other words, the technosphere matrix is very dense. Ecoinvent is stored as a [sparse matrix](http://docs.scipy.org/doc/scipy/reference/sparse.html), where data is only provided in about 1.5% of all possible locations - every other value is zero, and these zeros are not stored, only implied. However, the IO table has a fill rate of about 50%, meaning that we store every value in the matrix. The technosphere in ecoinvent 2.2 is about 4000 by 4000, but we only need to store about 40.000 numbers. The technosphere matrix is exiobase is about 8000 by 8000, but we store around 35.000.000 numbers.

We use a special backend for IO databases, as our standard storage mechanisms simply fall apart with such large data sets. You can see this [backend here](https://bitbucket.org/cmutel/brightway2-data/src/tip/bw2data/backends/iotable/__init__.py?at=2.0&fileviewer=file-view-default).

In [4]:
ex = Exiobase22Importer(fp)
ex.apply_strategies()
ex.write_database()

Extracting metadata
Extracting emissions
Extracting resources
Extracting main IO table
Extracted 163 datasets and many exchanges in 1.33 seconds
Aggregating `substances` and `extractions`
Processing technosphere

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 1.429 sec



Processing exchanges
Title: Writing activities to SQLite3 database:
  Started: 11/02/2015 22:05:08
  Finished: 11/02/2015 22:05:10
  Total time elapsed: 1.429 sec
  CPU %: 81.000000
  Memory %: 1.776457
Starting IO table write
Writing geomapping
Creating array - this will take a while...
On exchange number 1000000
On exchange number 2000000
On exchange number 3000000
On exchange number 4000000
On exchange number 5000000
On exchange number 6000000
On exchange number 7000000
On exchange number 8000000
On exchange number 9000000
On exchange number 10000000
On exchange number 11000000
On exchange number 12000000
On exchange number 13000000
On exchange number 14000000
On exchange number 15000000
On exchange number 16000000
On exchange number 17000000
On exchange number 18000000
On exchange number 19000000
On exchange number 20000000
On exchange number 21000000
On exchange number 22000000
On exchange number 23000000
On exchange number 24000000
On exchange number 25000000
On exchange number 

Free up some memory

In [ ]:
ex = None

# LCA calculations

We can now do an LCA. We first do this the standard way:

In [15]:
lca = LCA({Database("EXIOBASE 2.2").random(): 1}, methods.random())
lca.lci()
lca.lcia()
lca.score

Our technosphere matrix is sparse:

In [5]:
lca.technosphere_matrix

<7824x7824 sparse matrix of type '<class 'numpy.float64'>'
	with 35550884 stored elements in Compressed Sparse Row format>

And it takes a while to solve (versus less than one second for ecoinvent 2.2):

In [14]:
%timeit lca.solve_linear_system()

1 loops, best of 3: 13.2 s per loop


Free up some memory

In [ ]:
lca = None

However, we have a special LCA class that only does [dense technosphere matrices](https://bitbucket.org/cmutel/brightway2-calc/src/tip/bw2calc/dense_lca.py?at=default&fileviewer=file-view-default). If we use it, we will get better performance, because the linear solver assumes dense instead of sparse matrices:

In [3]:
dlca = DenseLCA({Database("EXIOBASE 2.2").random(): 1}, methods.random())
dlca.lci()

In [5]:
%timeit dlca.solve_linear_system()

1 loops, best of 3: 7.08 s per loop


Free up some more memory

In [7]:
print(dlca.tech_params.shape)
dlca.tech_params = None

(35557095,)

# Create aggregated processes

We can now create aggregated (so-called "system") processes for each activity in Exiobase. These aggregated proceses can be used in our normal sparse LCAs, but are terminated, i.e. we can't understand their background supply chains.

First, we create a new database.

In [31]:
aggregated_db = Database("EXIOBASE 2.2 aggregated")

In [13]:
type(aggregated_db)

bw2data.backends.peewee.database.SQLiteBackend

Now, we invert the EXIOBASE technosphere matrix.

This takes some minutes - around 4 on my laptop - so just be patient. It is helpful if there is plenty of free memory.

In [16]:
inverse = np.linalg.pinv(dlca.technosphere_matrix.todense())

With the inverse, we can calculated the aggregated inventories, and then write each aggregated process.

In [19]:
inventory = lca.biosphere_matrix * inverse
print(inventory.shape)

In [33]:
KEYS = (
    'exiobase_code',
    'group',
    'group_name',
    'location',
    'name',
    'synonym',
    'type',
    'unit'
)

data = {}

Only take each non-zero biosphere flow.

In [38]:
for ds in pyprind.prog_bar(IOTableBackend("EXIOBASE 2.2")):  # Database
    col = lca.activity_dict[ds.key]
    
    # Basic data
    data[("EXIOBASE 2.2 aggregated", ds['code'])] = {key: ds[key] for key in KEYS}
    # Exchanges
    data[("EXIOBASE 2.2 aggregated", ds['code'])]['exchanges'] = [{
        'type': 'biosphere',
        'amount': float(inventory[row, col]),
        'input': flow,
        'uncertainty type': 0
    } for flow, row in lca.biosphere_dict.items() if inventory[row, col]]

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 2.304 sec


In [40]:
aggregated_db.write(data)

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 34.066 sec


Title: Writing activities to SQLite3 database:
  Started: 11/03/2015 09:34:55
  Finished: 11/03/2015 09:35:29
  Total time elapsed: 34.066 sec
  CPU %: 84.300000
  Memory %: 9.030056


## We will now choose some sample LCA calculations in class